# **Data Preparation**

# Objectives

* Clean and remove non images.
* Prepare the dataset for further processes.
* Find average image shape for original images.
* Resize the images from the datasets.
* Split the datasets into Train, Validation and Test sets.

# Outputs

Resized images from the datasets for further processing

# Additional Comments

* No Additional Comments

In [1]:
import os
import shutil
import random
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
sns.set_style("white")

In [2]:
cwd= os.getcwd()

In [4]:
os.chdir(os.path.dirname(cwd))
print("You set a new current directory")

You set a new current directory


In [5]:
current_dir = os.getcwd()
current_dir

'/workspaces/PP5-Cherry-Leaves'

# Checks and Removes Corrupt Images and Image Types

In [8]:
import os
import tensorflow as tf

def remove_invalid_images(my_data_dir):
    image_extensions = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)

    for folder in folders:
        folder_path = os.path.join(my_data_dir, folder)
        
        if not os.path.isdir(folder_path):  
            continue  # Skip if not a folder

        i, j = 0, 0  # Counters

        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            if os.path.isfile(file_path):
                if not file_name.lower().endswith(image_extensions):  
                    os.remove(file_path)  # Remove non-image files
                    i += 1
                else:
                    try:
                        # Try opening with TensorFlow to verify it's an actual image
                        img = tf.io.read_file(file_path)
                        img = tf.io.decode_image(img, channels=3)
                        j += 1  # Valid image
                    except Exception as e:
                        print(f"Removing corrupted image: {file_path} - Error: {str(e)}")
                        os.remove(file_path)
                        i += 1

        print(f"Folder: {folder} - Valid Images: {j}, Removed: {i}")

# Example usage:
data_dir = '/workspaces/PP5-Cherry-Leaves/inputs/leaves_dataset/cherry-leaves'
remove_invalid_images(data_dir)

Folder: powdery_mildew - Valid Images: 2104, Removed: 0
Folder: healthy - Valid Images: 2104, Removed: 0
